In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
from models import networks
import torch

In [5]:
model = networks.TripletLossModel().cuda()

In [10]:
model(torch.ones(1, 3, 256, 256) * 0.7, torch.ones(1, 3, 256, 256) * 0.6, torch.ones(1, 3, 256, 256))

(tensor([[[[0.9809]],
 
          [[0.8990]],
 
          [[0.9128]],
 
          [[1.0065]],
 
          [[0.9278]],
 
          [[0.8603]],
 
          [[1.1050]],
 
          [[0.8572]],
 
          [[1.0613]],
 
          [[1.0436]],
 
          [[0.9247]],
 
          [[0.8919]],
 
          [[1.0322]],
 
          [[0.8852]],
 
          [[0.9173]],
 
          [[1.3189]],
 
          [[0.9821]],
 
          [[0.8969]],
 
          [[0.9678]],
 
          [[0.8984]],
 
          [[1.4163]],
 
          [[0.8421]],
 
          [[1.0850]],
 
          [[0.9075]],
 
          [[0.7728]],
 
          [[0.8419]],
 
          [[0.9853]],
 
          [[0.8910]],
 
          [[0.9263]],
 
          [[0.9836]],
 
          [[1.0639]],
 
          [[0.9626]],
 
          [[0.8427]],
 
          [[0.8882]],
 
          [[0.8447]],
 
          [[1.0974]],
 
          [[0.9388]],
 
          [[0.9998]],
 
          [[0.8946]],
 
          [[0.8167]],
 
          [[1.0570]],
 
          [[0.90

In [13]:
model.get_loss().backward()

In [6]:
model = 1

In [15]:
b = [3, 4]

In [16]:
a + b

[1, 2, 3, 3, 4]